## Homework 

In this assignment you'll be building a "dashboard" interface for the buildings dataset which provides the user with a linked view of this dataset.  The purpose of this assignment is to get some "hands-on" practice linking different facets of a dataset in a dashboard to provide the user a way to interactively browse their data.

Build a dashboard for the buildings data (see CSV file attached).

Left component:   
Grid heat map  
Rows are congressional district  
Columns are the governmental department (Agency Name) (note, the agency names might overlap with your plot -- there are ways around this or you can leave as is)  
Values are sum of total square footage for that set of criteria 
   
Right component:    
line plot     
x is the year   
y is total square footage acquired that year   
These two should be linked so that you can select cells and that will update the square footage plot.    
Things to think about:    
Can you keep the x and y ranges static on the line plot?    
Can you change the style?    


In your write-up please address:    

Any data transformations or rescalings you did for your plot
How you dealt with NaN's (empty entries) in your dataset
Any aesthetic choices you made (colors, layout, plot size, label size) and why, and what you'd like to experiment with if you had more time.
20 points for grid heatmap, 20 points for line plot, and 20 points for accurate linkage between the two.  40 points for writeup.

In [1]:
# import librarires 
import matplotlib 
import numpy as np
import pandas as pd 
import csv 
import bqplot
import ipywidgets

## Pre-processing 
* filtering the data used for visualization 
* for 'zero' values, replace to NaNs, drop NaNs. 

In [2]:
data=pd.read_csv('building_inventory.csv')

In [3]:
# Select the columns for use
data=data[['Congress Dist','Congressional Full Name','Year Acquired','Agency Name','Square Footage']]

In [4]:
len(data)

8862

In [6]:
# Along with NaNs, convert zeros also as NaNs. and drop the NaN values.
data=data.replace(0,np.nan)
data.dropna(inplace=True)
len(data)

8419

In [7]:
data

,Congress Dist,Congressional Full Name,Year Acquired,Agency Name,Square Footage
0,17.0,Cheri Bustos,1975.0,Department of Natural Resources,144.0
1,17.0,Cheri Bustos,2004.0,Department of Natural Resources,144.0
2,17.0,Cheri Bustos,2004.0,Department of Natural Resources,144.0
3,17.0,Cheri Bustos,2004.0,Department of Natural Resources,144.0
4,17.0,Cheri Bustos,2004.0,Department of Natural Resources,144.0
...,...,...,...,...,...
8851,17.0,Cheri Bustos,2018.0,Department of Transportation,1455.0
8852,4.0,Luis Gutierrez,2017.0,Department of Transportation,6720.0
8853,12.0,Mike Bost,2019.0,Department of Transportation,760.0
8854,12.0,Mike Bost,2019.0,Department of Transportation,760.0


## Left component 
* Grid Heat map 
* congressional District & Governmental department (agency)
* values are sum of total square footage for the set of criteria. 

In [8]:
#making heatmap data for Heat map 
heatmap_data=pd.pivot_table(data,index=['Congressional Full Name'],
                           values=['Square Footage'],
                           columns=['Agency Name'],
                           aggfunc=np.sum)

In [9]:
heatmap_data

Square Footage  \
Agency Name             Appellate Court / Fifth District   
Congressional Full Name                                    
Adam Kinzinger                                       NaN   
Bill Foster                                          NaN   
Bobby L. Rush                                        NaN   
Cheri Bustos                                         NaN   
Daniel William Lipinski                              NaN   
Danny K. Davis                                       NaN   
Darin M. LaHood                                      NaN   
Janice Schakowsky                                    NaN   
John Shimkus                                         NaN   
Luis Gutierrez                                       NaN   
Mike Bost                                        15124.0   
Mike Quigley                                         NaN   
Peter J. Roskam                                      NaN   
Randy Hultgren                                       NaN   
Robert Dold                                          NaN   
Robin Kelly                                          NaN   
Rodney L. Davis                                      NaN   
Tammy Duckworth                                      NaN   

                                                           \
Agency Name             Appellate Court / Fourth District   
Congressional Full Name                                     
Adam Kinzinger                                        NaN   
Bill Foster                                           NaN   
Bobby L. Rush                                         NaN   
Cheri Bustos                                          NaN   
Daniel William Lipinski                               NaN   
Danny K. Davis                                        NaN   
Darin M. LaHood                                       NaN   
Janice Schakowsky                                     NaN   
John Shimkus                                          NaN   
Luis Gutierrez                                        NaN   
Mike Bost                                             NaN   
Mike Quigley                                          NaN   
Peter J. Roskam                                       NaN   
Randy Hultgren                                        NaN   
Robert Dold                                           NaN   
Robin Kelly                                           NaN   
Rodney L. Davis                                   16400.0   
Tammy Duckworth                                       NaN   

                                                           \
Agency Name             Appellate Court / Second District   
Congressional Full Name                                     
Adam Kinzinger                                        NaN   
Bill Foster                                           NaN   
Bobby L. Rush                                         NaN   
Cheri Bustos                                          NaN   
Daniel William Lipinski                               NaN   
Danny K. Davis                                        NaN   
Darin M. LaHood                                       NaN   
Janice Schakowsky                                     NaN   
John Shimkus                                          NaN   
Luis Gutierrez                                        NaN   
Mike Bost                                             NaN   
Mike Quigley                                          NaN   
Peter J. Roskam                                       NaN   
Randy Hultgren                                        NaN   
Robert Dold                                           NaN   
Robin Kelly                                           NaN   
Rodney L. Davis                                       NaN   
Tammy Duckworth                                   43330.0   

                                                          \
Agency Name             Appellate Court / Third District   
Congressional Full Name                                    
Adam Kinzinger                       

## Linking 

In [11]:
# Lets first define the on_select (interactive function)
mySelectedLabel=ipywidgets.Label()
def on_selected(change):
    if len(change['owner'].selected)==1: 
        i,j=change['owner'].selected[0]
        h=heatmap_data.iloc[i,j]
        cong_name=heatmap_data.index[i]
       # print(cong_name)
        agency_name=heatmap_data.columns.values[j][1]
      #  print(agency_name)
        mySelectedLabel.value='Total Square Footage is : '+str(h)+'\n'+'Agency Name : '+str(agency_name)+'\n'+'Congressional District Name :'+str(cong_name)
        
        mask=(new_data['Congressional Full Name']==cong_name)&(new_data['Agency Name']==agency_name)
        line_plot.x=new_data['Year Acquired'][mask].tolist()
        line_plot.y=new_data['Square Footage Sum'][mask]
      #  print(h)

In [12]:
# (1) Scales: x/y, colors
col_sc = bqplot.ColorScale(scheme = "Reds")
x_sc = bqplot.LinearScale()
y_sc = bqplot.LinearScale()

# (2) Axis: x/y, colors
c_ax = bqplot.ColorAxis(scale = col_sc, 
                        orientation = 'vertical', 
                        side = 'right')
x_ax = bqplot.Axis(scale = x_sc,label='Agency')
y_ax = bqplot.Axis(scale = y_sc,label='Congressional Dist', 
                   orientation = 'vertical')

# (3) Marks: heatmap
heat_map = bqplot.GridHeatMap(color = heatmap_data,
                              scales = {'color': col_sc,
                                        'row': y_sc,
                                        'column': x_sc},
                              interactions = {'click': 'select'}, # make interactive on click of each box
                              anchor_style = {'fill':'blue'}, # to make our selection blue
                              selected_style = {'opacity': 1.0}, # make 100% opaque if box is selected
                              unselected_style = {'opacity': 0.8}) # make a little see-through if not

# (4) Link selection on heatmap to other things
heat_map.observe(on_selected, 'selected')

# (5) Paint heatmap canvas, don't display yet:
fig_heatmap = bqplot.Figure(marks = [heat_map], axes = [c_ax, y_ax, x_ax])

#fig_heatmap

## Right component 
* x: year 
* y: total square footage acquired that year 
* line plot 

### Refining data for line plot

In [13]:
#Refined the data, grouped by 'Year Acquired, Congress Dist, Agency Name' 
#to show total square Footage sum unique to year & agency & congdist for line plot
# (e.g. duplicated 2014, Congress Dist, Agency Square Footage => sum unique to 2014, Congress Dist, Agency Square Footage )

new_data=pd.DataFrame({'Square Footage Sum':data.groupby(['Year Acquired','Congressional Full Name','Agency Name'])['Square Footage'].sum()}).reset_index()

In [14]:
new_data

,Year Acquired,Congressional Full Name,Agency Name,Square Footage Sum
0,1753.0,Mike Bost,Historic Preservation Agency,1200.0
1,1802.0,Mike Bost,Historic Preservation Agency,4440.0
2,1810.0,Mike Bost,Historic Preservation Agency,4033.0
3,1832.0,Mike Bost,Southern Illinois University,120000.0
4,1837.0,John Shimkus,Historic Preservation Agency,10302.0
...,...,...,...,...
2427,2017.0,Luis Gutierrez,Department of Transportation,6720.0
2428,2018.0,Cheri Bustos,Department of Transportation,2910.0
2429,2018.0,Danny K. Davis,Department of Transportation,12000.0
2430,2018.0,Darin M. LaHood,Department of Transportation,2250.0


In [15]:
# (1) scales
x_sc=bqplot.LinearScale()
y_sc=bqplot.LinearScale()

# (2) Axis
ax_xcl = bqplot.Axis(label='Year Acquired', scale=x_sc)
ax_ycl = bqplot.Axis(label='Square Footage', scale=y_sc, 
                    orientation='vertical', side='left')

# (3) Marks
line_plot=bqplot.Lines(scales={'x':x_sc,'y':y_sc})

#
fig_line=bqplot.Figure(marks=[line_plot],axes=[ax_xcl,ax_ycl])

In [16]:
# since we know from last time we wanna make our figs a bit bigger:
fig_heatmap.layout.min_width='500px'
fig_line.layout.min_width='500px'

# side by side figures
figures = ipywidgets.HBox([fig_heatmap, fig_line])

# label on top
myDashboard = ipywidgets.VBox([mySelectedLabel, figures])
myDashboard

## Things to think about:
* Can you keep the x and y ranges static on the line plot?
> Yes, just as the range changes according to X, Y array, we can set the X arange & Y arange as masks and select the data according to the fixed range 
* Can you change the style?
> Yes, 1) we can change the color, 2) Line plot-> bar or histogram plot 

## Write Up 

* Any data transformations or rescalings you did for your plot    
> * Heatmap: Pivot table 
> ` heatmap_data=pd.pivot_table(data,index=['Congressional Full Name'],
values=['Square Footage'],columns=['Agency Name'], aggfunc=np.sum)`    
Since heatmap row should be congress dist, and columns Agency Name, corresponding to each square footage, pivot table was created as above. 
                           
> * Line plot : Grouped by unique `Year`, `Congress Dist`, `Agency Name`==> summed multiple square footage. 
Refined the data, grouped by `Year Acquired, Congress Dist, Agency Name` to show total square Footage sum unique to year & agency & congdist for line plot (e.g. duplicated 2014, Congress Dist, Agency Square Footage => sum unique to 2014, Congress Dist, Agency Square Footage )


* How you dealt with NaN's (empty entries) in your dataset
> At first, in the data preprocessing, filtered out all the NaNs for the selected columns. `Year, Square Footage, Congress Dist, Agency Name`. 

* Any aesthetic choices you made (colors, layout, plot size, label size) and why, and what you'd like to experiment with if you had more time. 
> Used color scale `Reds`, however we can choose variations of the color themes. 
> Label : Ipywidgets To show the details Ipywidget labels are used. 
> Plot size: in order to show two paralled plots, we fixed each plots as 500px. 